## Pregunta: T3 - Entre la primera y la última versión de Ubuntu, ¿han habido cambios en los pesos de los paquetes? ¿Las distros pesan más? Comparación de todos los paquetes comparativa con todas las versiones

### 1. Importaciones

In [7]:
import pandas as pd
from scipy import stats

### 2. Lectura

In [8]:
df = pd.read_csv("../../datawarehouse/processed-ubuntu-packages.csv")

### 3. Preprocesado

In [9]:
# Filtrar los datos para quedarnos solo con los paquetes que están presentes en más de una versión
paquetes = df['Package'].value_counts()[df['Package'].value_counts() > 1].index
df = df[df['Package'].isin(paquetes)]

# Convertir la columna Size a tipo numérico
df['Size'] = pd.to_numeric(df['Size'])

# Crear una columna con el número de versión de Ubuntu
df['VersionNumber'] = df['Distro-Version'].apply(lambda x: int(x.split('.')[0].replace('-', '')))


### 4. Calcular diferencia tamaño

In [10]:
# Calcular la diferencia de tamaño entre la primera y la última versión de Ubuntu para cada paquete
diff_sizes = df.groupby('Package')['Size'].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
diff_sizes.columns = ['Package', 'DiffSize']

# Realizar un test de hipótesis para determinar si hay evidencia suficiente para afirmar que las distros pesan más
t_stat, p_value = stats.ttest_1samp(diff_sizes['DiffSize'], 0)
if p_value < 0.05:
    print('Hay evidencia suficiente para afirmar que las distros pesan más.')
    print(f"El valor del estadístico de prueba t es {t_stat:.2f}, lo que indica que las diferencias en los tamaños de los paquetes son estadísticamente significativas.")
else:
    print('No hay evidencia suficiente para afirmar que las distros pesan más.')
    print(f"El p-value es {p_value:.2f}, lo que indica que no se puede rechazar la hipótesis nula de que no hay diferencias significativas en los tamaños de los paquetes entre la primera y la última versión de Ubuntu.")

Hay evidencia suficiente para afirmar que las distros pesan más.
El valor del estadístico de prueba t es 6.57, lo que indica que las diferencias en los tamaños de los paquetes son estadísticamente significativas.


In [11]:
# Calculamos la diferencia exacta de tamaño entre la primera y la última versión de Ubuntu para cada paquete
diff_sizes['DiffExact'] = df.groupby('Package')['Size'].apply(lambda x: x.iloc[-1] - x.iloc[0])

# Seleccionamos solo las columnas que nos interesan
diff_sizes = diff_sizes[['Package', 'DiffSize']]

# Mostramos el DataFrame con las diferencias de tamaño
print(diff_sizes)

               Package     DiffSize
0                  0ad    2079578.0
1             0ad-data  737301252.0
2      0ad-data-common       2832.0
3             0install     106924.0
4        0install-core     529940.0
...                ...          ...
77799    zypper-common     -63620.0
77800       zypper-doc        920.0
77801           zytrax      22792.0
77802      zziplib-bin      12600.0
77803             zzuf       9266.0

[77804 rows x 2 columns]
